## Voltage-based SOC estimation
This notebook implements two voltage-based SOC-estimation methods. The first one simply looks up cell terminal voltage under load in an OCV from SOC table. The second attempts to compensate for the effects of cell equivalent-series resistance.

In [1]:
import scipy.io

In [4]:
model = scipy.io.loadmat('../data/CellModel.mat', simplify_cells=True)['model']

In [6]:
model.keys()

dict_keys(['name', 'OCV0', 'OCVrel', 'SOC', 'OCV', 'SOC0', 'SOCrel', 'dOCV0', 'dOCVrel', 'temps', 'QParam', 'RCParam', 'RParam', 'GParam', 'Z0Param', 'R0Param', 'MParam'])

In [7]:
data = scipy.io.loadmat('../data/CellData.mat', simplify_cells=True)

In [8]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Sat Feb 16 06:25:50 2013',
 '__version__': '1.0',
 '__globals__': [],
 'time': array([    0,     1,     2, ..., 28507, 28508, 28509], dtype=uint16),
 'current': array([0., 0., 0., ..., 0., 0., 0.]),
 'voltage': array([4.17097378, 4.17158365, 4.17127848, ..., 3.57249636, 3.57273161,
        3.57231741]),
 'soc': array([1.        , 1.        , 1.        , ..., 0.05997534, 0.05997534,
        0.05997534])}